In [14]:
import pandas as pd
from modeling import feature_engineering, closest, agg_function,agg_diff_fun, save_object,load_object,get_best_params
import numpy as np
import reverse_geocoder as revgc
from sklearn.ensemble import RandomForestClassifier

In [15]:
df_train= pd.read_json('train.json')
df_test= pd.read_json('test.json')
metro= pd.read_csv('MetroStanice.csv')
df_train["training"]=1
df_test["training"]=0

df= pd.concat([df_train,df_test],sort=True).reset_index(drop=True)

mapping_features= pd.read_csv('feature_deduplication.csv')
mydict= dict(zip(mapping_features.original_feature, mapping_features.unique_feature))
#features and photos
df["len_features"] = df["features"].apply(len)
df["len_photos"] = df["photos"].apply(len)
df['features']= df['features'].apply(lambda x: [item.lower() for item in x]) 
df['features_txt']= df['features'].apply(lambda x: ' '.join(x))
df['features']=df['features'].apply(lambda x: [mydict[item] if item in mydict else item for item in x ])
#pocet features
#

#timefromepoch
import datetime
#df['tstamp'] =df['created'].apply(lambda x:  datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').timestamp())
#Datumove
#df["date"]=  df['created'].str[:7]
df['created']=pd.to_datetime(df["created"])
#df['day'] = df['created'].dt.day
#df['days']=(df["created"]-min(df["created"])).apply(lambda x: x.days)
df['dayofweek'] = df['created'].dt.dayofweek
#cylicke
df['hr'] = df['created'].dt.hour
df['mnth'] = df['created'].dt.month
df['nwords_description']= df['description'].apply(lambda x: x.count(' ')+1)
df['price_digits']=df['price'].apply(lambda x: len(str(x)))
df['price'] = np.log(df['price'])

# GEO
coordinates= df[["latitude","longitude"]].values
coordinates_tuple=tuple((x[0],x[1]) for x in coordinates)
adresy=revgc.search(coordinates_tuple)
df=pd.concat([df,pd.DataFrame(adresy)],axis=1)


#label encoding
event_dictionary ={'low' : 1, 'medium' : 2, 'high' : 3} 
df['interest_integer'] =df['interest_level'].map(event_dictionary) 

#interakcie zakladna intuicia
df['bed+bath']=df['bedrooms']+df['bathrooms']
df['price/bed']=df['price']/(df['bedrooms']+1e-16)
df['price/bed+bath']=df['price']/(df['bed+bath']+1e-16)

training='training'
label='interest_integer'

metro=metro.loc[:,['lat','lon']]
metro=metro.to_dict('records')
dfmetro=df.loc[:,['latitude','longitude']]
dfmetro=dfmetro.to_dict('records')
x=[]
for i in dfmetro:
    x.append(closest(metro, i))
df['metro']=x


#drop columns
df=df.drop(columns=['photos','created','listing_id','lat','lon','latitude','longitude'])

In [3]:
agg_dict={agg_function: ['mean','median','sum','std'],agg_diff_fun: ['mean', 'median']}
training='training'
label='interest_integer'
pca_list=[1,2,3,4,5]
num_features=50
clf = RandomForestClassifier(max_depth=5,criterion='gini', random_state=0,n_estimators=150,n_jobs=-1)
df_out,columns,good_encoders,dp_table,df_tokenizer=feature_engineering(df,training,label,num_features,agg_dict,pca_list,clf =clf)
save_object([df_out,columns,good_encoders,dp_table,df_tokenizer],clf.__class__.__name__+'_thesis_v_rve')

2020-05-28 20:55:28.208360


100%|██████████| 2/2 [00:07<00:00,  3.79s/it]


after numeric feature selection:  ['price', 'len_photos', 'hr', 'price/bed'] 0.6776139370308577


  0%|          | 0/7 [00:00<?, ?it/s]

after TOP N features:  ['price', 'len_photos', 'hr', 'price/bed', 'hardwood', 'no fee'] 0.6758182856319486


...

3 <function stemming_tokenizer at 0x7fb0cf5c55f0> 1
3 <function stemming_tokenizer at 0x7fb0cf5c55f0> 2
3 <function stemming_tokenizer at 0x7fb0cf5c55f0> 3
3 <function stemming_tokenizer at 0x7fb0cf5c55f0> 4
3 <function stemming_tokenizer at 0x7fb0cf5c55f0> 5


/home/martas/anaconda3/envs/mirko/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


3 <modeling.LemmaTokenizer object at 0x7fb0aeb99a50> 1
3 <modeling.LemmaTokenizer object at 0x7fb0aeb99a50> 2
3 <modeling.LemmaTokenizer object at 0x7fb0aeb99a50> 3
3 <modeling.LemmaTokenizer object at 0x7fb0aeb99a50> 4
3 <modeling.LemmaTokenizer object at 0x7fb0aeb99a50> 5
3 None 1
3 None 2
3 None 3
3 None 4


100%|██████████| 3/3 [18:05<00:00, 361.77s/it]

3 None 5
after text feature tokenization plus PCA:  ['price', 'hr', 'no fee', 'sum_bathrooms_by_building_id', 'std_price_by_building_id', 'price+price_digits', 'bathrooms+price/bed+bath', 'price+price/bed+bath', 'len_photos+price_digits', 'building_id', 'manager_id', 'name'] 0.6159339788809312
2020-05-28 22:50:36.823854


In [ ]:
agg_dict={agg_function: ['mean','median','sum','std'],agg_diff_fun: ['mean', 'median']}
training='training'
label='interest_integer'
pca_list=[1,2,3,4,5]
num_features=50
clf = RandomForestClassifier(max_depth=5,criterion='gini', random_state=0,n_estimators=150,n_jobs=-1)
df_out,columns,good_encoders,dp_table,df_tokenizer=feature_engineering(df,training,label,num_features,agg_dict,pca_list,clf =clf)
save_object([df_out,columns,good_encoders,dp_table,df_tokenizer],clf.__class__.__name__+'_thesis')

In [ ]:
parameters=pd.read_csv('RandomForestClassifier_grid_results_extra.csv')
best_params=get_best_params(parameters,'score')
best_params
agg_dict={agg_function: ['mean','median','sum','std'],agg_diff_fun: ['mean', 'median']}
training='training'
label='interest_integer'
pca_list=[1,2,3,4,5]
num_features=50
clf = RandomForestClassifier( random_state=0,n_jobs=-1,**best_params)
df_out,columns,good_encoders,dp_table,df_tokenizer=feature_engineering(df,training,label,num_features,agg_dict,pca_list,clf =clf)
save_object([df_out,columns,good_encoders,dp_table,df_tokenizer],clf.__class__.__name__+'_thesis_best_grid_params_extra')

2020-06-02 20:17:24.079344


100%|██████████| 2/2 [00:39<00:00, 19.94s/it]


after numeric feature selection:  ['bathrooms', 'price', 'len_features', 'len_photos', 'hr', 'nwords_description', 'price/bed+bath', 'metro'] 0.6295682664973432


  0%|          | 0/7 [00:00<?, ?it/s]

after TOP N features:  ['bathrooms', 'price', 'len_features', 'len_photos', 'hr', 'nwords_description', 'price/bed+bath', 'metro', 'laundry', 'elevator', 'hardwood', 'cats allowed', 'concierge', 'no fee', 'fitness center', 'pre war', 'exclusive', 'simplex', 'reduced fee', 'furnished'] 0.6124358184149072


 43%|████▎     | 3/7 [1:34:39<1:57:39, 1764.83s/it]